# COVID-19 WES / WGS Analysis: Hail Outline
#### Written by Kumar Veerapen, PhD (veerapen@broadinstitute.org)
#### Date: June 2nd, 2020
#### Objective: To run sample, genotype, and variant QC on WES / WGS data generated from the COVID-19 Global Host Genetics Initiative  

Hail is an open-source Python library that simplifies genomic data analysis. It provides powerful, easy-to-use data science tools that can be used to interrogate even biobank-scale genomic data (e.g UK Biobank, TopMed, FinnGen, and Biobank Japan). Get more information on usage from https://hail.is/

Notes on the outline of this script can be found on : https://docs.google.com/document/d/1X_qjplH8T4BJXSeMQ_sBfQUTiu_kAisicOqGb6B8hcM/edit

Code snippets adapted from [SCHEMA](https://github.com/astheeggeggs/BipEx/tree/master/scripts_BipEx/QC_BipEx) (Schizophrenia Exome Sequencing Meta-analysis), SPARK (Simons Foundation Powering Autism Research for Knowledge) WES pipelines (Tarjinder Singh, PhD and Kyle Statterstrom, PhD), and [gnomAD](https://github.com/broadinstitute/gnomad_qc/tree/79b12c7cfe04034c2b28f1a3ac0160d2fa5d81c1/gnomad_qc) (Genome Aggregation Database by Konrad Karczewski, PhD), and supplementary scripts by lead analysts at the Analytic and Translational Genetics Unit ([ATGU](http://www.atgu.mgh.harvard.edu/)), Massachusetts General Hospital and the Broad Institute of MIT and Harvard .

## Getting started

View Hail documentation on Installation: https://hail.is/docs/0.2/getting_started.html#installation
We highly recommend the use of a cloud platform for your analytics e.g. Google cloud (which is European General Data Protection Regulation (GDPR) compliant) 

## Set up your python environment
In addition to Hail, we import a few methods from the Hail plotting library.  Import additional libraries that you would see fit in your analytics

In [ ]:
import hail as hl
from hail.plot import output_notebook, show
import bokeh
from pprint import pprint

Now we initialize Hail and set up plotting to display inline in the notebook.

In [ ]:
hl.init()
output_notebook()

The demonstration materials are designed to work on a small (~20MB) downsampled chunk of the public 1000 Genomes dataset.
It is possible to call command-line utilities from Jupyter by prefixing a line with a `!`:

In [ ]:
! ls -1 resources/

## Import data from VCF

The [Variant Call Format (VCF)](https://en.wikipedia.org/wiki/Variant_Call_Format) is a common file format for representing genetic data collected on multiple individuals (samples).

Hail's [import_vcf](https://hail.is/docs/0.2/methods/impex.html#hail.methods.import_vcf) function can read this format.

However, VCF is a text format that is easy for humans to read, but very inefficient to process on a computer. 

The first thing we do is import (`import_vcf`) and convert the `VCF` file into a Hail native file format. This is done by using the `write` method below. The resulting file is **much** faster to process because it is scalable and easily parallelizable.

Picking partition number is somewhat arbitrary. We recommend a ratio of 1:1 = WGS:partitions or 10:1 = WES:partitions


**Note**: We HIGHLY recommend using GRCh38 for your callset. In the following example, the 1kg toy example is in GRCh37 and would therefore need to be edited to suit your dataset.

In [ ]:
hl.import_vcf('resources/1kg.vcf.bgz', min_partitions=4, reference_genome='GRCh37', force_bgz=True).write('resources/1kg.mt', overwrite=True)

### Read 1KG into Hail

We represent genetic data as a Hail [`MatrixTable`](https://hail.is/docs/0.2/overview/matrix_table.html), and name our variable `mt` to indicate this.

In [ ]:
mt = hl.read_matrix_table('resources/1kg.mt')

We highly recommend exploring your matrixTable if this is your first time using Hail using functions like `show`, `summarize`, or `count`. One of our personal interactive favourites is:

In [ ]:
mt.describe(widget=True)

You can show individual fields like the sample ID (`s`), 

In [ ]:
mt.s.show()

the locus (`locus`)

In [ ]:
mt.locus.show()

or the called genotype (`GT`):

In [ ]:
mt.GT.show()

`summarize` Prints (potentially) useful information about any field or object:

In [ ]:
mt.DP.summarize()

In [ ]:
mt.AD.summarize()

`MatrixTable.count` returns a tuple with the number of rows (variants) and number of columns (samples).

In [ ]:
mt.count()

### Count before splitting multi-allelics.
Based on the analytical plans as of May 2020, there are no plans to include non-SNPs in this example script. We would recommend to split multiallelic variants into biallelic variants. First asses what are you dealing with in number of samples and variants:

In [ ]:
n = mt.count()

pprint('n samples:')
print(n[1])
pprint('n variants:')
print(n[0])

In [ ]:
hl.summarize_variants(mt)

In [ ]:
mt = hl.split_multi_hts(mt)

In [ ]:
hl.summarize_variants(mt)

### Annotate matrixTable with sample and phenotype annotation

#### Integrate sample information

This is a text file containing phenotype information:

In [ ]:
! head resources/1kg_annotations.txt

We can import it as a [Hail Table](https://hail.is/docs/0.2/overview/table.html) with [hl.import_table](https://hail.is/docs/0.2/methods/impex.html?highlight=import_table#hail.methods.import_table).

We call it "sa" for "sample annotations".

In [ ]:
sa = hl.import_table('resources/1kg_annotations.txt', 
                      impute=True, 
                      key='s')

In [ ]:
sa.show()

In [ ]:
sa.summarize()

### Add sample metadata into our 1KG `MatrixTable`

In Hail, annotate methods refer to adding new fields.

MatrixTable's annotate_cols adds new column (sample) fields.
MatrixTable's annotate_rows adds new row (variant) fields.
MatrixTable's annotate_entries adds new entry (genotype) fields.
Table's annotate adds new row fields.

It just takes one line:

In [ ]:
mt = mt.annotate_cols(pheno = sa[mt.s])

In Hail, `annotate` methods refer to **adding new fields**. 

 - `MatrixTable`'s `annotate_cols` adds new column (**sample**) fields.
 - `MatrixTable`'s `annotate_rows` adds new row (**variant**) fields.
 - `MatrixTable`'s `annotate_entries` adds new entry (**genotype**) fields.
 - `Table`'s `annotate` adds new row fields.

In the above cell, we are adding a new column (**sample**) field called "pheno". This field should be the values in our table `sa` associated with the sample ID `s` in our `MatrixTable` - that is, this is performing a **join**.

You should think of this in much the same way - for each column of `mt`, we are looking up the fields in `sa` using the sample ID `s`.

Let's look at where does this go into the `MatrixTable`

## Part 2 from COVID-19 doc: 
### 2.0 Sample QC

We'll start with examples of sample QC.

Hail has the function [hl.sample_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc) to compute a list of useful statistics about samples from sequencing data. This function adds a new column field, `sample_qc`, with the computed statistics.

**Click the link** above to see the documentation, which lists the fields and their descriptions.

In [ ]:
mt = hl.sample_qc(mt)

Visualize the distribution of `Mean DP` (`DP` = Read Depth) to `Call Rate`:

In [ ]:
p = hl.plot.scatter(x=mt.sample_qc.dp_stats.mean,
                    y=mt.sample_qc.call_rate,
                    xlabel='Mean DP',
                    ylabel='Call Rate',
                    hover_fields={'ID': mt.s},
                    size=8)
show(p)

In [ ]:
mt.count()

We recommend a call rate filter of 97%

In [ ]:
mt = mt.filter_cols(mt.sample_qc.call_rate >= 0.97)

In [ ]:
p = hl.plot.scatter(x=mt.sample_qc.dp_stats.mean,
                    y=mt.sample_qc.call_rate,
                    xlabel='Mean DP',
                    ylabel='Call Rate',
                    hover_fields={'ID': mt.s},
                    size=8)
show(p)

In [ ]:
mt.describe(widget=True)

Note the number of variants removed

In [ ]:
mt.count()

#### 2.0.2    Sex Imputation

We suggest inferring for sex using the Hail function [impute_sex](https://hail.is/docs/0.2/methods/genetics.html?highlight=impute_sex#hail.methods.impute_sex). This function should be performed on common biallelic SNPs (AF > 0.05) with a high callrate (callrate > 0.97). Suggested thresholds for this function include the following. We would also recommend plotting the data to observe data is within reasonable limits of thresholds set: 
`aaf_threshold: 0.05`
`female_threshold: 0.5`
`male_threshold: 0.75`

First filter for high quality calls for sex QC

In [ ]:
mt = mt.filter_rows((hl.len(mt.alleles) == 2) & hl.is_snp(mt.alleles[0], mt.alleles[1]) &
                            (hl.agg.mean(mt.GT.n_alt_alleles()) / 2 > 0.001) &
                            (hl.agg.fraction(hl.is_defined(mt.GT)) > 0.97))

In [ ]:
mt.count()

Imputing sex with thresholds defined above and write it into a Hail table

In [ ]:
imputed_sex = hl.impute_sex(mt.GT,aaf_threshold=0.05, female_threshold=0.5, male_threshold=0.75)


In [ ]:
imputed_sex.show()

Annotate matrixtable with the imputed sex

In [ ]:
mt = mt.annotate_cols(impute_sex = imputed_sex[mt.s])

#### 2.0.3    Additional filters
Recommended filters removing samples that are  
Mean coverage < 20.0
Ambiguous sex
Aneuploids
Call rate < 97

Annotate for sex aneuploids. 

You can use aneuploid definitions to filte out ambigious samples or you can use the Hail `impute_sex` binary of true/false to do so.

In [ ]:
mt = mt.annotate_cols(aneuploid= ((mt.impute_sex.f_stat >= 0.5) ) | (hl.is_missing(mt.impute_sex.f_stat)) | 
                      ((mt.impute_sex.f_stat >= 0.4) & (mt.impute_sex.f_stat <= 0.6) ) ,
        sex_aneuploidy=(mt.impute_sex.f_stat < 0.4) )

In [ ]:
mt.count()

Using `impute_sex` true/false to determine if you need to remove a sample of ambigious sex

In [ ]:
mt = mt.filter_cols( (mt.sample_qc.call_rate >= 0.97) &
                    (mt.sample_qc.dp_stats.mean > 20) & (hl.is_defined(mt.impute_sex.is_female))  )

**OR** using your sex aneuploid annotated column

In [ ]:
mt = mt.filter_cols( (mt.sample_qc.call_rate >= 0.97) &
                    (mt.sample_qc.dp_stats.mean > 20) & (hl.is_defined(mt.aneuploid))  )

In [ ]:
mt.count()

#### 2.0.4    Relatedness filter
Samples can be filtered to remove one of each pair of related samples using Hail's [maximal_independent_set](https://hail.is/docs/0.2/methods/misc.html?highlight=maximal_independent_set#hail.methods.maximal_independent_set) (uses model free relatedness estimation via PC-Relate). We suggest filtering for samples with second-degree relatedness or higher, where one of each pair of samples with a kinship coefficient of > 0.088 can be removed.

Run PC-relate and compute pairs of closely related individuals:
Note that the filtered kinship coefficient is already listed as the recommended 0.088

In [ ]:
pca_eigenvalues, pca_scores, pca_loadings = hl.hwe_normalized_pca(mt.GT, k=10, compute_loadings=False)

In [ ]:
 relatedness_ht = hl.pc_relate(mt.GT, min_individual_maf=0.01, scores_expr=pca_scores[mt.col_key].scores,
                                      block_size=4096, min_kinship=0.1, statistics='all')
       

In [ ]:
pairs = relatedness_ht.filter(relatedness_ht['kin'] > 0.088)

In [ ]:
related_samples_to_remove = hl.maximal_independent_set(pairs.i, pairs.j, False)

Starting from the above pairs, prune individuals from a dataset until no close relationships remain:

In [ ]:
mt.count()

In [ ]:
mt = mt.filter_cols(hl.is_defined(related_samples_to_remove[mt.col_key]), keep=False)

In [ ]:
mt.count()

#### 2.0.5    Population Ancestry Inference
Principal component analysis (PCA) is a very general statistical method for reducing high dimensional data to a small number of dimensions which capture most of the variation in the data. Hail has the function [pca](https://hail.is/docs/0.2/methods/stats.html#hail.methods.pca) for performing generic PCA.

PCA typically works best on normalized data (e.g. mean centered). Hail provides the specialized function [hwe_normalized_pca](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca) which first normalizes the genotypes according to the Hardy-Weinberg Equilibium model.

In [ ]:
pca_eigenvalues, pca_scores, pca_loadings = hl.hwe_normalized_pca(mt.GT, compute_loadings=True)

The pca function returns three things:
* **eigenvalues**: an array of doubles
* **scores**: a table keyed by sample
* **loadings**: a table keyed by variant

The **loadings** are the *principal directions*, each of which is a weighted sum of variants (a "virtual variant"). By default, `hwe_normalized_pca` gives us 10 principal directions.

In [ ]:
mt = mt.annotate_cols(pca = pca_scores[mt.s])

Take the first 4 PCs from the PCA table, and add the population information for each sample from our dataset.

In [ ]:
ht = pca_scores.select(PC1=pca_scores.scores[0],
                       PC2=pca_scores.scores[1],
                       PC3=pca_scores.scores[2],
                       PC4=pca_scores.scores[3])
ht = ht.annotate(pheno = sa[ht.s])

The five populations present in this dataset are `AFR`, `AMR`, `EAS`, `EUR`, and `SAS`. They are three-letter codes from the 1000 Genomes project denoting the [super population of each sample](https://www.internationalgenome.org/category/population/).

##### Visualize!

Let's plot several combinations of the first four principal components (PCs) against each other. This will help us visualize the population structure of the dataset, and allow us to try identify our samples with different population ancestry clusters. Note that since the plots generated by the `hl.plot` module use the `bokeh` plotting library internally, we can use `bokeh` functions like `gridplot` to arrange  plots.

In [ ]:
p1 = hl.plot.scatter(ht.PC1, ht.PC2, xlabel='PC1', ylabel='PC2', label=ht.pheno.super_population, size=6)
p2 = hl.plot.scatter(ht.PC1, ht.PC3, xlabel='PC1', ylabel='PC3', label=ht.pheno.super_population, size=6)
p3 = hl.plot.scatter(ht.PC2, ht.PC4, xlabel='PC2', ylabel='PC4', label=ht.pheno.super_population, size=6)


show(bokeh.layouts.gridplot([[p1], [p2], [p3]]))

Based on your visualization, you can then choose to cluster your samples based on  ancestry inference using the following code structure suggestion 

In [ ]:
check(ht.annotate(
    unmasked = hl.case()
        .when((ht.PC2 > 0.2) & (ht.PC1 < 0), 'EAS')
#         .when(..., 'AFR')
#         .when(..., 'AMR')
#         .when(..., 'EUR')
#         .when(..., 'SAS')
        .default(ht.pheno.super_population)
))

#### 2.0.5	Population Ancestry Inference  
##### THIS SUBSECTION IS STILL UNDER CONSTRUCTION

To increase accuracy of inferring population ancestry, we recommend selecting an approach based on study ascertainment: 


a) _If population ascertained is relatively homogenous:_

We recommend performing PCA projection of the exome data onto the gnomAD population principal components and then to use a random forest classifier trained on gnomAD ancestry labels to assign ancestry to the exome samples. (Konrad to provide loadings and RF for gnomAD without training of model)


b) _If population ascertained is relatively heterogeneous (multi-ancestry):_

We recommend using a hybrid approach that would first be PCA projection expressed in point a). Secondly, to account for highly admixed samples, we recommend that a from-scratch PCA be performed on the exome dataset using an unsupervised learning/clustering method, e.g. HDBSCAN. Using this hybrid method, any sample that was assigned to a cluster using the from-scratch PCA is given that cluster as their ancestry assignment in order to preserve the substructure observed compared to a projection PCA method. Any sample that was not assigned to a cluster was given the label from the PCA project and random forest classification.
Methods outside of the above mentioned are welcome, if the user has good enough reason to choose otherwise.


#### 2.0.6	Outlier Detection

Utilizing the Hail [sample_qc](https://hail.is/docs/0.2/methods/genetics.html?highlight=sample_qc#hail.methods.sample_qc) method, we suggest removing outliers that deviate from the median and median absolute deviation (MAD) (non-parametric equivalent for mean and standard deviation) for the following metrics. It is also important to note that these outlier detection metrics below would need to be stratified by population ancestry (and sequencing platform) determined from subsection 2.0.5: 


`n_snp`: Number of SNP alternate alleles

`r_ti_tv`: Transition/transversion ratio

`r_insertion_deletion`: Insertion/Deletion allele ratio

`n_insertion`: Number of insertion alternate alleles

`n_deletion`: Number of deletion alternate alleles

`r_het_hom_var`: Heterozygous/homozygous call ratio


Using medians and median absolute deviation (MAD), we can estimate removal of outliers. 

The following code blocks:

1) is an outline of what can be done for **separately** for each population ancestry and sequencing platform.

2) look at the `n_snp` metric and needs to be interrogated (*and replaced in script below*) for `r_ti_tv`, `r_insertion_deletion`, `n_insertion`, `n_deletion`, and `r_het_hom_var`.

In [ ]:
metric_values = hl.agg.collect(mt.sample_qc.n_snp)
metric_median = hl.median(metric_values)
metric_mad = 1.4826 * hl.median(hl.abs(metric_values - metric_median))
outlier_metric=hl.struct( median=metric_median,
            mad=metric_mad,
            upper=metric_median + 4 * metric_mad,
            lower=metric_median - 4 * metric_mad)


mt = mt.annotate_globals(metrics_stats=mt.aggregate_cols(outlier_metric))

In [ ]:
mt.globals.metrics_stats.show()

Apply filter for the selected metric. **Remember** that this step needs to be done for each 

1) population

2) sequencing platform

3) each memtric (`n_snp`, `r_ti_tv`, `r_insertion_deletion`, `n_insertion`, `n_deletion`, and `r_het_hom_var`)


In [ ]:
mt=mt.filter_cols( (mt.sample_qc.n_snp <= mt.metrics_stats.upper) |
            (mt.sample_qc.n_snp >=  mt.metrics_stats.lower) )

In [ ]:
mt.count()

### 2.1	Variant Quality Control 
Upon completion of the Sample QC described in section 2.0, exomes should then be processed for Variant QC that is further elaborated in this section 3.0. We recommend applying a PASS filter using the Variant Quality Score Recalibration (VQSR) metric.  

Hail has the function [hl.variant_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc) to compute a list of useful statistics about **variants** from sequencing data.

In [ ]:
mt = hl.variant_qc(mt)

In [ ]:
show(hl.plot.cdf(mt.variant_qc.call_rate))

In [ ]:
mt.describe(widget=True)

In [ ]:
mt = mt.annotate_rows(fail_VQSR = hl.len(mt.filters) == 0)

In [ ]:
mt.filter_rows(mt.fail_VQSR).count_rows()

In [ ]:
mt.filters.show()

In [ ]:
### Must insert LCR file if you choose to filter for Low complexity regions
LCR_intervals = hl.import_locus_intervals(<LCRs>, reference_genome='GRCh38')
mt = mt.annotate_rows(in_LCR = hl.is_defined(LCR_intervals[mt.locus]))

Get information about the number of variants that were excluded.

In [ ]:
in_LCR = mt.filter_rows(mt.in_LCR).count_rows()
print('n variants in low complexity regions:')
pprint(in_LCR)

In [ ]:
mt = mt.filter_rows(mt.in_LCR, keep=False)

Annotate variants with flag indicating if they failed VQSR.
In this toy example, there is no information on VQSR, so everything is removed. Be weary of your data!


In [ ]:
mt = mt.annotate_rows(fail_VQSR = hl.len(mt.filters) != 0)

Get information about the number of variants that were excluded.

In [ ]:
fail_VQSR = mt.filter_rows(mt.fail_VQSR).count_rows()
print('n variants failing VQSR:')
pprint(fail_VQSR)

In [ ]:
mt = mt.filter_rows(mt.fail_VQSR, keep=False)

Filter out the invariant rows.

In [ ]:
mt = mt.filter_rows((mt.qc.AF[0] > 0.0) & (mt.qc.AF[0] < 1.0))

### 2.2	Genotype Quality Control
High quality genotypes can be filtered when applying the following thresholds. We would also recommend performing call rate filtering separately for cases and controls: differential missingness is a typical source of false positives:

`GQ` >= 20

`DP` >= 10

`AB` >= 0.25 (for each allele in heterozygous calls)

In [ ]:
#create an allele balance annotation
mt= mt.annotate_entries(AB = (mt.AD[1] / hl.sum(mt.AD) ))

In [ ]:
#set filter condition for AB
filter_condition_ab = ((mt.GT.is_hom_ref() & (mt.AB <= 0.1)) |
                        (mt.GT.is_het() & (mt.AB >= 0.25) & (mt.AB <= 0.75)) |
                        (mt.GT.is_hom_var() & (mt.AB >= 0.9)))
fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')


In [ ]:
mt = mt.filter_entries( (mt.GQ>=20) &
                 (mt.GQ >= 10) &
                 ((mt.GT.is_hom_ref() & (mt.AB <= 0.1)) |
                        (mt.GT.is_het() & (mt.AB >= 0.25) & (mt.AB <= 0.75)) |
                        (mt.GT.is_hom_var() & (mt.AB >= 0.9)))) 

### 2.3     Functional Annotation
Variants can be annotated using the Variant Effect Predictor (VEP) annotation as implemented in Hail [annotation_db](https://hail.is/docs/0.2/annotation_database_ui.html) either using the function `hail.vep` or using [gnomAD utils](https://broadinstitute.github.io/gnomad_methods/examples/vep.html) using the default parameters for GRCh38 (including LOFTEE). In addition, for downstream gene-based tests, we recommend grouping variants into genes by canonical transcripts in Ensembl Gene ID and/or HGNC symbols with the following annotations: 

**pLoF**: High-confidence LoF variants [LOFTEE](https://github.com/konradjk/loftee), including stop-gained, essential splice, and frameshift variants, filtered according to a set of first principles as described on the Github repo and [gnomAD](https://www.biorxiv.org/content/10.1101/531210v2)


**Missense | Low-confidence(LC)**: Missense variants are grouped with in-frame insertions and deletions, as well as low-confidence LoF variants (filtered out by LOFTEE). The latter have a frequency spectrum consistent with missense variation, and affect a set of amino acids in a similar fashion (e.g. a frameshift in the final exon).

**synonymous**: All synonymous variants in the gene (control set).


*Additional VEP or machine learning method annotations* available e.g. ‘splice_region_variant’ or  [kipoi](https://kipoi.org) repository (ref: Julien Gagneur).


**The following code will _ONLY_ work on the Google cloud platform**

In [ ]:
#assuming that you have loaded the latest version of Hail
mt = hl.vep(mt)

Additional annotations such as CADD scores.
Options can be found [here](https://hail.is/docs/0.2/annotation_database_ui.html#id1) :

In [ ]:
db = hl.experimental.DB()
mt = db.annotate_rows_db(mt, "CADD")

_Additional note_

If you are using your own annotation database or would like other annotations, you are able to easily do it with Hail tables.

You would need to have the annotations in tabular formats and follow instructions of importing annotation tables as a Hail table i.e. subsection above ### Annotate matrixTable with sample and phenotype annotation and subsequently annotate your variants i.e. subsection above ### Add sample metadata into our 1KG `MatrixTable`. The difference between the annotation subsection is that instead of using the `annotate_cols` function, you would use `annotate_rows`
